In [31]:
import numpy as np

In [32]:
# Load the data
data = np.loadtxt("databp.csv", delimiter=',', skiprows=0)  # skiprows=0 to not skip the header
print(data)

[[0. 0.]
 [0. 1.]
 [0. 3.]
 [1. 2.]
 [0. 1.]
 [0. 1.]
 [0. 2.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 3.]
 [2. 4.]
 [0. 1.]
 [0. 3.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 2.]
 [0. 3.]
 [0. 1.]
 [0. 2.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 3.]
 [0. 1.]
 [1. 1.]
 [1. 2.]
 [0. 2.]
 [0. 0.]
 [1. 2.]
 [1. 2.]
 [1. 1.]
 [1. 1.]
 [2. 1.]
 [1. 1.]
 [1. 2.]
 [1. 2.]
 [2. 0.]
 [2. 0.]
 [4. 0.]
 [4. 0.]
 [1. 3.]
 [3. 2.]
 [4. 1.]
 [4. 0.]
 [6. 3.]
 [3. 2.]
 [2. 1.]
 [2. 0.]
 [2. 3.]
 [2. 3.]
 [1. 2.]
 [2. 2.]
 [2. 2.]
 [0. 1.]
 [0. 2.]
 [0. 1.]
 [0. 1.]
 [1. 3.]
 [1. 2.]
 [1. 1.]
 [0. 0.]
 [0. 2.]
 [0. 2.]
 [1. 2.]
 [2. 0.]
 [3. 1.]
 [2. 1.]
 [4. 1.]
 [2. 2.]
 [1. 1.]
 [0. 1.]
 [0. 2.]
 [0. 1.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 1.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 1.]
 [1. 4.]
 [0. 2.]
 [0. 3.]
 [2. 1.]
 [1. 2.]
 [4. 1.]
 [2. 3.]
 [4. 2.]
 [2. 1.]
 [0. 3.]
 [1. 2.]
 [1. 3.]
 [2. 2.]
 [1. 1.]
 [2. 2.]
 [1. 3.]
 [1. 3.]
 [0. 0.]
 [0. 0.]
 [0. 2.]
 [1. 1.]
 [1. 3.]
 [2. 1.]
 [0. 1.]
 [0. 1.]
 [1. 3.]
 [0. 1.]
 [1. 2.]
 

In [33]:
# Center the data
def center_data(data):
    means = data.mean(axis=0)
    Y = data - means
    return Y

In [34]:
centered_data = center_data(data)
print(centered_data)

[[-1.02  -1.738]
 [-1.02  -0.738]
 [-1.02   1.262]
 [-0.02   0.262]
 [-1.02  -0.738]
 [-1.02  -0.738]
 [-1.02   0.262]
 [-1.02  -1.738]
 [-1.02  -0.738]
 [-1.02  -0.738]
 [-1.02   1.262]
 [ 0.98   2.262]
 [-1.02  -0.738]
 [-1.02   1.262]
 [-1.02  -0.738]
 [-1.02   0.262]
 [-1.02   0.262]
 [-1.02   0.262]
 [-1.02   1.262]
 [-1.02  -0.738]
 [-1.02   0.262]
 [-1.02  -0.738]
 [-1.02  -0.738]
 [-1.02  -0.738]
 [-1.02   1.262]
 [-1.02  -0.738]
 [-0.02  -0.738]
 [-0.02   0.262]
 [-1.02   0.262]
 [-1.02  -1.738]
 [-0.02   0.262]
 [-0.02   0.262]
 [-0.02  -0.738]
 [-0.02  -0.738]
 [ 0.98  -0.738]
 [-0.02  -0.738]
 [-0.02   0.262]
 [-0.02   0.262]
 [ 0.98  -1.738]
 [ 0.98  -1.738]
 [ 2.98  -1.738]
 [ 2.98  -1.738]
 [-0.02   1.262]
 [ 1.98   0.262]
 [ 2.98  -0.738]
 [ 2.98  -1.738]
 [ 4.98   1.262]
 [ 1.98   0.262]
 [ 0.98  -0.738]
 [ 0.98  -1.738]
 [ 0.98   1.262]
 [ 0.98   1.262]
 [-0.02   0.262]
 [ 0.98   0.262]
 [ 0.98   0.262]
 [-1.02  -0.738]
 [-1.02   0.262]
 [-1.02  -0.738]
 [-1.02  -0.73

In [35]:
# Fit VAR model
def fit_VAR(data, p):
    n, k = data.shape
    X = center_data(data)
    
    # Construct the regressor matrix Xprime
    Xprime = np.zeros((n - p, k * p))
    X_target = np.zeros((n - p, k))
    
    for t in range(p, n):
        Xprime_row = []
        for lag in range(1, p + 1):
            Xprime_row.extend(X[t - lag])
        Xprime[t - p, :] = Xprime_row
        X_target[t - p] = X[t]
    
    # Estimate coefficients using OLS
    beta = np.linalg.inv(Xprime.T @ Xprime) @ Xprime.T @ X_target
    # Predicting X_hat
    X_hat = Xprime @ beta
    residuals = X[p:] - X_hat
    
    return residuals, beta

In [36]:
residuals, beta = fit_VAR(data, 3)
print(beta)

# changement de forme pour adapter et comparer a l'output de la fonction de fit statsmodels
beta_reshaped = beta.T.reshape(2, 3, 2)
print(beta_reshaped)

[[ 0.51044174 -0.05972736]
 [ 0.03491085  0.21350878]
 [ 0.1292632   0.13479687]
 [ 0.00349909  0.05604906]
 [ 0.02002635 -0.02783168]
 [-0.00629847 -0.00335769]]
[[[ 0.51044174  0.03491085]
  [ 0.1292632   0.00349909]
  [ 0.02002635 -0.00629847]]

 [[-0.05972736  0.21350878]
  [ 0.13479687  0.05604906]
  [-0.02783168 -0.00335769]]]


In [8]:
x1 = np.random.rand(5,1)
print(x1)
x2 = np.random.rand(5,1)
print(x2)
xc = np.concatenate((x1, x2), axis = 1)
print(xc, np.shape(xc))
xc[3,:] = xc[1,0], xc[2,0]
print(xc[3:,0])


[[0.32514028]
 [0.59745533]
 [0.23169089]
 [0.0109393 ]
 [0.51107997]]
[[0.20949875]
 [0.26208052]
 [0.51979602]
 [0.64102952]
 [0.57917436]]
[[0.32514028 0.20949875]
 [0.59745533 0.26208052]
 [0.23169089 0.51979602]
 [0.0109393  0.64102952]
 [0.51107997 0.57917436]] (5, 2)
[0.59745533 0.51107997]


In [23]:
n = 497
print(data)
X1 = data[p+1:, 0]
X2 = data[p:, 1]
#Xs = np.zeros((n,2))
#Xs[:,0] = X1
#Xs[:,1] = X2
#print(f"xs:{Xs}")
print(np.shape(X1))

[[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 2.]
 [1. 0.]
 [3. 2.]
 [0. 0.]
 [3. 1.]
 [1. 1.]
 [1. 1.]
 [3. 0.]
 [3. 1.]
 [3. 0.]
 [1. 0.]
 [0. 0.]
 [0. 2.]
 [2. 2.]
 [3. 2.]
 [2. 0.]
 [2. 1.]
 [0. 1.]
 [1. 2.]
 [1. 3.]
 [0. 4.]
 [0. 3.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [2. 1.]
 [3. 2.]
 [3. 2.]
 [3. 1.]
 [1. 1.]
 [1. 1.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [2. 1.]
 [2. 1.]
 [0. 2.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 3.]
 [0. 0.]
 [0. 1.]
 [0. 1.]
 [0. 0.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [0. 1.]
 [0. 0.]
 [0. 2.]
 [0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 1.]
 [0. 2.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 0.]
 [1. 0.]
 [2. 1.]
 [2. 1.]
 [2. 3.]
 [3. 0.]
 [3. 1.]
 [0. 0.]
 [1. 1.]
 [2. 1.]
 [4. 1.]
 [4. 1.]
 [1. 2.]
 [3. 3.]
 [1. 2.]
 [0. 1.]
 [0. 2.]
 [1. 3.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [2. 1.]
 [3. 1.]
 [0. 1.]
 [0. 4.]
 [1. 1.]
 [0. 5.]
 [0. 3.]
 [0. 2.]
 [0. 1.]
 [1. 0.]
 [2. 1.]
 [0. 1.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 

In [37]:
print(residuals)

[[ 6.00503992e-01  7.62630317e-02]
 [-8.75726594e-01 -7.59241250e-01]
 [-4.43541170e-01 -6.77492255e-01]
 [-3.37103706e-01  5.37827660e-01]
 [-3.58286683e-01 -1.70687050e+00]
 [-2.91964066e-01 -3.35901992e-01]
 [-3.13578270e-01 -4.33954965e-01]
 [-3.29674303e-01  1.50328059e+00]
 [ 1.60680246e+00  2.07962072e+00]
 [-1.45599003e+00 -1.12653145e+00]
 [-5.79802543e-01  1.07561276e+00]
 [-4.14354814e-01 -8.54642837e-01]
 [-3.30374006e-01  3.94540169e-01]
 [-3.45689738e-01  2.99844887e-01]
 [-3.61785771e-01  2.37080446e-01]
 [-3.55487298e-01  1.24043814e+00]
 [-3.90398151e-01 -9.73070644e-01]
 [-3.24075534e-01  3.97897860e-01]
 [-3.45689738e-01 -7.00155113e-01]
 [-3.26874919e-01 -5.49410773e-01]
 [-3.17077358e-01 -4.90004023e-01]
 [-3.23375831e-01  1.50663829e+00]
 [-3.93197535e-01 -9.20379277e-01]
 [ 6.69625994e-01 -6.05459831e-01]
 [ 1.78779379e-01  5.73081031e-01]
 [-9.97991615e-01  2.18060002e-01]
 [-5.11075316e-01 -1.86988474e+00]
 [ 6.94308058e-01  6.95287382e-01]
 [ 1.41069142e-01  4

In [38]:
# Function to estimate VAR-ARMA parameters
def estimate_VAR_ARMA(data, p, residuals): #p = ordre du VAR sur les donnees centrees, necessaire pour les retards et valeurs manquantes des residus
    T, N = data.shape
    assert N == 2, "This implementation is for bivariate VAR only."
    print(f"T = {T}")
    n = T - p  # Number of rows after adjusting for lags
    Xprime = np.zeros((n-1, 5)) #n lignes pour les pbs, 3 = 1 cst + 2 termes retard X_t-1 + 2 terme retard et-1
    Xprime[:, 0] = 1  # colonne de la constante 
    print(f"n = {n}")
    for t in range(0, n-1):
        Xprime[t, 1:3] = data[p+t, :] #pour t= 0 le terme le plus lointain dans nos données (celui de retard 1) est le (p+1)eme element de nos données, et le premier qui a un terme de retard correspondant
        Xprime[t, 3:5] = residuals[t, :]
    print(Xprime)
    X1 = data[p+1:, 0]
    X2 = data[p+1:, 1]
    Xs = np.zeros((n-1,2))
    Xs[:,0] = X1
    Xs[:,1] = X2

    # Estimate coefficients using OLS
    beta = np.linalg.inv(Xprime.T @ Xprime) @ Xprime.T @ Xs

    Phi = np.zeros((2, 2))
    Theta = np.zeros((2, 2))
    omega = [beta[0, 0], beta[0, 1]]
    Theta[0, :] = beta[3,0], beta[4,0]
    Theta[1, :] = beta[3,1], beta[4,1]
    Phi[0, :] = beta[1,0], beta[1,1]
    Phi[1, :] = beta[2,0], beta[2,1]
    
    return omega, Theta, Phi

In [39]:
# Assuming `data` and `residuals` are already defined
p = 3
omega, Theta, Phi = estimate_VAR_ARMA(data, p, residuals)
print("Omega:\n", omega)
print("Theta:\n", Theta)
print("Phi:\n", Phi)

T = 500
n = 497
[[ 1.          1.          2.          0.60050399  0.07626303]
 [ 1.          0.          1.         -0.87572659 -0.75924125]
 [ 1.          0.          1.         -0.44354117 -0.67749225]
 ...
 [ 1.          1.          5.         -0.22690043  2.62467051]
 [ 1.          0.          1.         -1.14697566 -1.53159246]
 [ 1.          1.          6.          0.53223369  4.18554994]]
Omega:
 [0.23465160195974022, 1.0382414738637697]
Theta:
 [[-0.23186028  0.01570848]
 [-0.18039404 -0.11806852]]
Phi:
 [[0.74365324 0.12172813]
 [0.01673798 0.3317491 ]]


In [40]:
A = -Theta
print(f"A = {A}")
B = Phi+Theta
print(f"B = {B}")

A = [[ 0.23186028 -0.01570848]
 [ 0.18039404  0.11806852]]
B = [[ 0.51179296  0.13743662]
 [-0.16365607  0.21368058]]
